In [1]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
import numpy as np
import pandas as pd
from time import time, sleep
from datetime import datetime

In [2]:
# Create directories for saving images
import os
left_images_dir = 'left_images'
right_images_dir = 'right_images'
depth_images_dir = 'depth_images'
os.makedirs(left_images_dir, exist_ok=True)
os.makedirs(right_images_dir, exist_ok=True)
os.makedirs(depth_images_dir, exist_ok=True)

In [3]:
class SimpleDataCollector(Node):
    def __init__(self):
        super().__init__('simple_data_collector')

        self.bridge = CvBridge()
        self.last_timestamp = 0
        self.sync_rate = 1/100
        
        self.left_image = None
        self.right_image = None
        self.depth_image = None

        self.left_time = 0
        self.right_time = 0
        self.depth_time = 0

        self.left_image_sub = self.create_subscription(Image, '/cam_1/zed_node/left/image_rect_color', self.left_image_callback, 10)
        self.right_image_sub = self.create_subscription(Image, '/cam_1/zed_node/right/image_rect_color', self.right_image_callback, 10)
        self.depth_image_sub = self.create_subscription(Image, '/cam_1/zed_node/depth/depth_registered', self.depth_image_callback, 10)
        
    def synchronization(self):
        #if self.left_image is not None and self.right_image is not None and self.depth_image is not None:
        if self.left_time == self.right_time == self.depth_time:

            cv_left = self.bridge.imgmsg_to_cv2(self.left_image, desired_encoding='bgr8')
            cv_right = self.bridge.imgmsg_to_cv2(self.right_image, desired_encoding='bgr8')
            cv_depth = self.bridge.imgmsg_to_cv2(self.depth_image, desired_encoding='passthrough')

            #cv_depth = np.where(np.isnan(cv_depth), 4, cv_depth)
            #cv_depth = np.where(np.isinf(cv_depth), 0.3, cv_depth)

            left_filename = f"{left_images_dir}/frame_{self.left_time}.png"
            right_filename = f"{right_images_dir}/frame_{self.right_time}.png"
            depth_filename = f"{depth_images_dir}/frame_{self.depth_time}.npy"

            cv2.imwrite(left_filename, cv_left)
            cv2.imwrite(right_filename, cv_right)
            np.save(depth_filename, cv_depth)
            #cv_depth = pd.DataFrame(cv_depth)
            #cv_depth.to_csv(depth_filename)
            # cv2.imwrite(depth_filename, cv_depth)
            
            print('Current time is:' + str(self.left_time))
            interval = self.left_time - self.last_timestamp
            print('Time   interval:' + str(interval))

            self.last_timestamp = self.left_time
            
    def left_image_callback(self, msg):
        self.left_image = msg
        self.left_time = self.left_image.header.stamp.sec + self.left_image.header.stamp.nanosec / 1e9
        self.synchronization()

    def right_image_callback(self, msg):
        self.right_image = msg
        self.right_time = self.right_image.header.stamp.sec + self.right_image.header.stamp.nanosec / 1e9
        self.synchronization()

    def depth_image_callback(self, msg):
        self.depth_image = msg
        self.depth_time = self.depth_image.header.stamp.sec + self.depth_image.header.stamp.nanosec / 1e9
        self.synchronization()

In [4]:
rclpy.init()
simple_data_collector = SimpleDataCollector()
rclpy.spin(simple_data_collector)

Current time is:1707792922.2827163
Time   interval:1707792922.2827163
Current time is:1707792922.2994082
Time   interval:0.016691923141479492
Current time is:1707792922.3160212
Time   interval:0.016613006591796875
Current time is:1707792922.3327672
Time   interval:0.016746044158935547
Current time is:1707792922.3495333
Time   interval:0.016766071319580078
Current time is:1707792922.3660262
Time   interval:0.016492843627929688
Current time is:1707792922.3829482
Time   interval:0.0169219970703125
Current time is:1707792922.3995612
Time   interval:0.016613006591796875
Current time is:1707792922.4162793
Time   interval:0.016718149185180664
Current time is:1707792922.4329052
Time   interval:0.01662588119506836
Current time is:1707792922.4496593
Time   interval:0.016754150390625
Current time is:1707792922.4662843
Time   interval:0.016624927520751953
Current time is:1707792922.4830194
Time   interval:0.016735076904296875
Current time is:1707792922.4996483
Time   interval:0.01662898063659668
C

KeyboardInterrupt: 